In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/AI ML project

/content/drive/MyDrive/ColabNotebooks/AI ML project


In [ ]:
import numpy as np
import pandas as pd
import re
import string
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import pickle

In [ ]:
with open('static/model/model.pickle', 'rb') as f:
  model = pickle.load(f)

In [ ]:
def remove_punctuation(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, '')
  return text

In [ ]:
#open stopwords for remove stopwords from the dataset
with open('static/model/corpora/stopwords/english', 'r') as file:
  sw = file.read().splitlines()

In [ ]:
vocab = pd.read_csv("static/model/vocabulary.txt", header=None) #../statis/model/vocabulary.txt
token = vocab[0].tolist()

In [ ]:
def preprocessing(text):
  data = pd.DataFrame(text, columns=["tweet"])
  #convert uppercase to lowercase
  data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))

  #remove links
  data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))

  #remove punctuation
  data["tweet"] = data["tweet"].apply(remove_punctuation)

  #remove numbers
  data["tweet"] = data["tweet"].str.replace('\d+', '', regex=True)

  #remove stopwords
  data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

  #stemming
  data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

  return data["tweet"]


In [ ]:
def vectorizer(dataset , vocabulary):
  vectorized_list = []

  for sentence in dataset:
    sentence_list = np.zeros(len(vocabulary))

    for i in range(len(vocabulary)):
      if vocabulary[i] in sentence.split():
        sentence_list[i] = 1

    vectorized_list.append(sentence_list)

  vectorized_list_new = np.array(vectorized_list, dtype=np.float32)

  return vectorized_list_new

In [ ]:
def get_prediction(vectorized_txt):
  prediction = model.predict(vectorized_txt)
  if prediction > 0:
    return "positive"
  else:
    return "negative"

In [ ]:
txt = "this is very good . i love this so much"
preprocessed_txt = preprocessing([txt])
vectorized_txt = vectorizer(preprocessed_txt, token)
prediction = get_prediction(vectorized_txt)
print(prediction)

negative


In [ ]:
model.predict(vectorized_txt)

array([1])